# Cricket Match H2H Duel

In [1]:
import numpy as np
import pandas as pd
import json

match_id = 13569002
# Api given script for collect json format data using python
import http.client

conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}

conn.request("GET", "/api/cricket/match/13569002/duel", headers=headers)

res = conn.getresponse()
data = res.read()

# end api script

data_json = json.loads(data.decode("utf-8"))

# Save to a file
with open("api_data.json", "w", encoding="utf-8") as f:
    json.dump(data_json, f, indent=4)

print("Data saved as api_data.json")

Data saved as api_data.json


In [1]:
import numpy as np
import pandas as pd
import json
import http.client
import time  
# List of match IDs

match_ids = [
    13265827, 13265828, 13265829, 13265830, 13265831, 13265832, 13265833, 
    13265834, 13265836, 13265837, 13265838, 13265840, 13558575, 13558576, 13569002
]

# API connection setup
conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}

# Parent dictionary to store all match data
parent_json = {}

# Loop through each match ID
for match_id in match_ids:
    try:
        # Make API request
        conn.request("GET", f"/api/cricket/match/{match_id}/duel", headers=headers)
        res = conn.getresponse()
        data = res.read()
        
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        
        # Store JSON in the parent dictionary with match_id as key
        parent_json[f"match_duel_{match_id}"] = data_json

        print(f"Data collected for Match ID {match_id}")

    except Exception as e:
        print(f"Error fetching data for Match ID {match_id}: {e}")

    time.sleep(1)  # Optional: Delay to prevent rate limits

# Save all match data in a single JSON file
with open("all_match_duel_data.json", "w", encoding="utf-8") as f:
    json.dump(parent_json, f, indent=4)

print("All match data saved in all_match_duel_data.json")

Data collected for Match ID 13265827
Data collected for Match ID 13265828
Data collected for Match ID 13265829
Data collected for Match ID 13265830
Data collected for Match ID 13265831
Data collected for Match ID 13265832
Data collected for Match ID 13265833
Data collected for Match ID 13265834
Data collected for Match ID 13265836
Data collected for Match ID 13265837
Data collected for Match ID 13265838
Data collected for Match ID 13265840
Data collected for Match ID 13558575
Data collected for Match ID 13558576
Data collected for Match ID 13569002
All match data saved in all_match_duel_data.json


## Loading Json file

In [9]:
with open("all_match_duel_data.json", "r", encoding="utf-8") as f:
    all_match_duel = json.load(f)
all_match_duel

{'match_duel_13265827': {'teamDuel': {'homeWins': 3,
   'awayWins': 7,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265828': {'teamDuel': {'homeWins': 0,
   'awayWins': 10,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265829': {'teamDuel': {'homeWins': 2,
   'awayWins': 8,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265830': {'teamDuel': {'homeWins': 7,
   'awayWins': 3,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265831': {'teamDuel': {'homeWins': 2,
   'awayWins': 8,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265832': {'teamDuel': {'homeWins': 3,
   'awayWins': 7,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265833': {'teamDuel': {'homeWins': 6,
   'awayWins': 4,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265834': {'teamDuel': {'homeWins': 2,
   'awayWins': 6,
   'draws': 0},
  'managerDuel': None},
 'match_duel_13265836': {'teamDuel': {'homeWins': 7,
   'awayWins': 3,
   'draws': 0},
  'managerDuel': None},


# Depth json to flatten_json

In [10]:
# Function for Convert a nested json dictionary to flatten json or dictionary
def flatten_json(nested_json, parent_key='', sep='_'):

    items = []
    
    # If the value is a dictionary
    if isinstance(nested_json, dict):
        for key, value in nested_json.items():
            new_key = f"{parent_key}{sep}{key}" if parent_key else key
            items.extend(flatten_json(value, new_key, sep=sep).items())
    
    # If the value is a list
    elif isinstance(nested_json, list): 
        for index, item in enumerate(nested_json):
            new_key = f"{parent_key}{sep}{index}" if parent_key else str(index)
            items.extend(flatten_json(item, new_key, sep=sep).items())
            
    # If the value is neither dict nor list
    else:
        items.append((parent_key, nested_json))

    return dict(items)

# Flatten Json to Row column

In [11]:
def process_json_file(data):
    try:
        # Normalize JSON data
        if isinstance(data, list):  # JSON is an array of objects
            flat_data = [flatten_json(item) for item in data]
        elif isinstance(data, dict):  # JSON is a single dictionary
            flat_data = [flatten_json(data)]
        else:
            raise ValueError("Unsupported JSON format")

        # Convert to DataFrame
        df = pd.DataFrame(flat_data)
        return df


    except json.JSONDecodeError:
        print("Error: Invalid JSON file.")
    except Exception as e:
        print(f"Error: {str(e)}")

In [14]:
all_match_h2h_duel = pd.DataFrame()
for i in all_match_duel.keys():
    match_id = int(i.split('_')[-1])
    data = all_match_duel[i]
    match_h2h_duel = process_json_file(flatten_json(data))
    match_h2h_duel['match_id'] = match_id
    all_match_h2h_duel = pd.concat([all_match_h2h_duel, match_h2h_duel], ignore_index=True)

In [16]:
all_match_h2h_duel.to_excel('all_match_h2h_duel.xlsx', index=False)